In [4]:
from roboflow import Roboflow
import os
from dotenv import load_dotenv
load_dotenv()

rf = Roboflow(api_key="M3Zhnyp687xQRSVg6vct")
# project = rf.workspace("fyp-3bwmg").project("reloc2-den7l")
# version = project.version(1)
# dataset = version.download("yolov8")

# dataset 2
project = rf.workspace("roboflow-jvuqo").project("basketball-court-detection-2")
version = project.version(19)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to basketball-court-detection-2-19 in yolov8:: 100%|██████████| 2932/2932 [00:00<00:00, 11086.86it/s]


In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n-pose.pt")

augumentations = {#"hsv_h": 0.0,
                  #"hsv_s": 0.0,
                  #"hsv_v": 0.0,
                  "translate": 0.0,
                  "scale": 0.0,
                  #"shear": 0.0,
                  #"perspective": 0.0,
                  "fliplr": 0.0,
                  "flipud": 0.0,
                  "mosaic": 0.0,
                  "mixup": 0.0,
                  "cutmix": 0.0,
                  #"erasing": 0.0,
                  "copy_paste": 0.0}

model.train(data="basketball-court-detection-2-19/data.yaml",
            epochs=100,
            device=0,
            plots=True,
            # imgsz=640,
            **augumentations,
            batch=8,)

New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.224 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=basketball-court-detection-2-19/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-pose.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=tra

In [2]:
import numpy as np

full_court_x_m = 28
full_court_y_m = 15
delta_y_0_1_m = 0.9
box_y_m = 4.8
box_x_m = 5.79

full_court_x = 1
full_court_y = full_court_y_m/full_court_x_m

delta_y_0_1 = delta_y_0_1_m/full_court_y_m
box_y       = box_y_m/full_court_x_m
box_x       = box_x_m/full_court_x_m

reference_keypoints = np.array([[0, 0],
                                [0, delta_y_0_1],
                                [0, full_court_y/2 - box_y/2],
                                [0, full_court_y/2 + box_y/2],
                                [0, full_court_y - delta_y_0_1],
                                [0, full_court_y],
                                [full_court_x/2, full_court_y],
                                [full_court_x/2, 0],
                                [box_x, full_court_y/2 - box_y/2],
                                [box_x, full_court_y/2 + box_y/2],
                                [full_court_x, full_court_y],
                                [full_court_x, full_court_y - delta_y_0_1],
                                [full_court_x, full_court_y/2 + box_y/2],
                                [full_court_x, full_court_y/2 - box_y/2],
                                [full_court_x, delta_y_0_1],   
                                [full_court_x, 0],
                                [full_court_x - box_x, full_court_y/2 - box_y/2],
                                [full_court_x - box_x, full_court_y/2 + box_y/2]]) 

In [5]:
from ultralytics import YOLO
from court_stitch import load_frames
import cv2

# --- Load YOLO model ---
# For keypoints (pose): use "yolo11n-pose.pt" or your trained weights
model = YOLO("../runs/pose/train8/weights/best.pt")  # or "runs/pose/train/weights/best.pt"

# --- Source (can be image, folder, video, webcam index) ---
source = "../input_videos/video_1.mp4"   # could also be "0" for webcam or "image.jpg"

frames = load_frames(source, sample_rate=1)
# --- Run Inference ---
for frame in frames:
    results = model.predict(
        source=frame,     # path to file, directory, or camera stream
        conf=0.5,          # confidence threshold
        # imgsz=1280,
    )

    keypoints = results[0].keypoints

    if keypoints is None or len(keypoints.xy) == 0:
        print("No detections in this frame — skipping.")
        continue

    frame_keypoints = keypoints.xy.cpu().numpy()[0]
    conf = keypoints.conf.cpu().numpy()[0]

    conf_thresh = 0.3
    kp_scale = 600

    valid_ref_kp = reference_keypoints[conf > conf_thresh] * kp_scale
    valid_frame_kp = frame_keypoints[conf > conf_thresh]

    if len(valid_frame_kp) >= 4:
        tmp = frame.copy()
        for (x, y) in valid_frame_kp:
            cv2.circle(tmp, (int(x), int(y)), 4, (0,255,0), -1)

        cv2.imshow("Frame points", tmp)

        blank = np.ones((int(full_court_y * kp_scale), int(full_court_x * kp_scale), 3), np.uint8)*255
        for (x, y) in valid_ref_kp:
            cv2.circle(blank, (int(x), int(y)), 4, (0,0,255), -1)
        
        cv2.imshow("Ref points", blank) 

        H, _ = cv2.findHomography(valid_frame_kp, valid_ref_kp, cv2.RANSAC, 5)
        if H is not None:
            img2_warped = cv2.warpPerspective(frame, H, (int(full_court_x * 600), int(full_court_y * 600)))
            print(valid_frame_kp)
            cv2.imshow('Warped Image', img2_warped)
            
        key = cv2.waitKey(0)
    
cv2.destroyAllWindows()


[INFO] Loaded 117 frames.

0: 736x1280 1 basketball, 5.3ms
Speed: 1.8ms preprocess, 5.3ms inference, 2.0ms postprocess per image at shape (1, 3, 736, 1280)
[[     445.59      247.37]
 [     425.51       258.2]
 [        327      322.92]
 [     179.52      416.05]
 [     21.428      519.73]
 [     615.73      624.98]
 [      652.6      279.89]
 [     711.84      348.38]
 [     623.89      447.72]]

0: 736x1280 1 basketball, 4.7ms
Speed: 1.9ms preprocess, 4.7ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)
[[     439.85      248.06]
 [      419.2      259.32]
 [     320.92      324.82]
 [     172.03      418.49]
 [     615.02      628.69]
 [     648.76      277.09]
 [     711.21      348.83]
 [     622.86      449.41]]

0: 736x1280 1 basketball, 4.6ms
Speed: 1.9ms preprocess, 4.6ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)
[[     444.09      247.01]
 [     422.25      257.01]
 [     323.36       322.9]
 [     170.03      418.23]
 [     4.0423  